# Assignment 8

Develop a model for 20 news groups dataset from scikit-learn. Select 20% of data for test set.

Develop metric learning model with siamese network [3 points] and softmax loss [3 points] (from seminar). Use KNN and LSH (any library for approximate nearest neighbor search) for final prediction after the network was trained. [2 points]

! Remember, that LSH gives you a set of neighbor candidates, for which you have to calculate distances to choose top-k nearest neighbors.

Your quality metric = accuracy score [2 points if acc > 0.8 ]

In [0]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [0]:
type(newsgroups_train.data), len(newsgroups_train.data), len(newsgroups_train.target)

In [0]:
newsgroups_train.target_names

In [0]:
target_vals = set(newsgroups_train.target)

In [0]:
df = {x: [] for x in set(newsgroups_train.target)}
for idx, anchor in enumerate(newsgroups_train.data):
    target = newsgroups_train.target[idx]
    df[target] = df[target] + [anchor]

In [0]:
len(df[0])

480

In [0]:
type(target_vals)

set

In [0]:
import pandas as pd
import random

In [0]:
# form new dataset:
#    1) anchor: новостная статья
#    2) pos: случайная другая новостная статья из той же группы
#    3) neg: случайная другая новостная статья из любой другой группы

In [0]:
new_df = {}
for idx, anchor in enumerate(newsgroups_train.data):
    target = newsgroups_train.target[idx]
    pos = random.choice(df[target])
    another_group = random.choice(list(target_vals - set([target])))
    neg = random.choice(df[another_group])
    new_df[idx] = {'anchor': anchor, 'pos': pos, 'neg': neg, 'target': target}
new_df = pd.DataFrame.from_dict(new_df, orient = 'index')

In [0]:
new_df.head()

,anchor,pos,neg,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,From: rhorwell@crab.atc.boeing.com (Roland Far...,From: valo@cvtstu.cvt.stuba.cs (Valo Roman)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,From: Thomas Kephart <kephart@snowhite.eeap.cw...,From: pmoloney@maths.tcd.ie (Paul Moloney)\nSu...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,From: pritchet@cs.scarolina.edu (Ronald W. Pri...,From: hal@cco.caltech.edu (Hal Finney)\nSubjec...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,From: ukrphil@prlhp1.prl.philips.co.uk (M.J.Ph...,From: brandt@cs.unc.edu (Andrew Brandt)\nSubje...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,Subject: Diffs to sci.space/sci.astro Frequent...,From: rmohns@vax.clarku.edu\nSubject: RE: Can ...,14


In [0]:
# Compute two different representation for each token.
# Each representation is a linear weighted combination for the
# 3 layers in ELMo (i.e., charcnn, the outputs of the two BiLSTM))

In [0]:
!pip install allennlp
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file, weight_file, 2, dropout=0)

     |████████████████████████████████| 7.6MB 2.8MB/s 
     |████████████████████████████████| 30.9MB 1.4MB/s 
     |████████████████████████████████| 256kB 60.0MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 163kB 60.7MB/s 
     |████████████████████████████████| 245kB 63.2MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 133kB 61.9MB/s 
     |████████████████████████████████| 204kB 66.0MB/s 
     |████████████████████████████████| 92kB 13.5MB/s 
     |████████████████████████████████| 3.2MB 61.3MB/s 
     |████████████████████████████████| 2.1MB 45.1MB/s 
     |████████████████████████████████| 1.0MB 50.3MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.15.0-cp36-cp36m-linux_x86_64.whl size=3319882 sha256=d7674147d2b45109853b3d68ea083651aeaa327bfb4ebde395bdbc428fb4d7e2
  Stored in directory: /root/.cache/pip/wheels/57/63/2e/da89cfe1ba08550bd7262d5d9c027edc313980c3b85b3b0a38
  Cre

100%|██████████| 374434792/374434792 [00:23<00:00, 16144482.16B/s]


In [0]:
# преобразовать pandas dataframe в pytorch dataset для цикла по batch

In [0]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(new_df, test_size=0.2, random_state=42, shuffle=True)

xa_train = batch_to_ids(df_train.anchor.values)
xp_train = batch_to_ids(df_train.pos.values)
xn_train = batch_to_ids(df_train.neg.values)
y_train = tt.from_numpy(df_train.target.values).float()

xa_val = batch_to_ids(df_val.anchor.values)
xp_val = batch_to_ids(df_val.pos.values)
xn_val = batch_to_ids(df_val.neg.values)
y_val = tt.from_numpy(df_val.is_duplicate.values).float()

In [0]:
from torch.utils.data import Dataset, DataLoader

batch_size = 32 # try 16
train_loader = DataLoader(TensorDataset(xa_train, xp_train, xn_train, y_train), batch_size=batch_size)
val_loader = DataLoader(TensorDataset(xa_val, xp_val, xn_val, y_val), batch_size=batch_size)

In [0]:
def _train_epoch(model, iterator, optimizer, curr_epoch):

    model.train()

    running_loss = 0

    n_batches = len(iterator)
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        loss = model(batch)
        loss.backward()
        optimizer.step()

        curr_loss = loss.data.cpu().detach().item()
        
        loss_smoothing = i / (i+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss

        iterator.set_postfix(loss='%.5f' % running_loss)

    return running_loss

In [0]:
def _test_epoch(model, iterator):
    model.eval()
    epoch_loss = 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            loss = model(batch)
            epoch_loss += loss.data.item()

    return epoch_loss / n_batches

In [0]:
def nn_train(model, train_iterator, valid_iterator, optimizer, n_epochs=100,
          scheduler=None, early_stopping=0):

    prev_loss = 100500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss = _train_epoch(model, train_iterator, optimizer, epoch)
        valid_loss = _test_epoch(model, valid_iterator)

        valid_loss = valid_loss
        print('validation loss %.5f' % valid_loss)

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)

In [0]:
model = Tripletnet(elmo)
optimizer = optim.Adam(model.parameters())
nn_train(model, train_loader, val_loader, optimizer, n_epochs=2)

In [0]:
import torch
import torch.nn as nn

def triplet_loss(anchor_embed, pos_embed, neg_embed):
    return F.cosine_similarity(anchor_embed, neg_embed) - F.cosine_similarity(anchor_embed, pos_embed)
    
    
class Tripletnet(nn.Module):
    def __init__(self, elmo, fc_num = 1024, hidden_dim = 128):
        super(Tripletnet, self).__init__()
        self.elmo = elmo
        self.criterion = criterion
        
        self.fc = nn.Linear(fc_num*3, hidden_dim)
        
    def branch(self, x):
        x = self.elmo(x)['elmo_representations'] # vector representation
        x = tt.cat(x, dim=-1)
        x = x.mean(dim=1)
        x = self.fc(x)
        return x

    def forward(self, batch):
        anchor, pos, neg = batch
        anchor = self.branch(anchor)
        pos = self.branch(pos)
        neg = self.branch(neg)
        
        return triplet_loss(anchor, pos, neg)

# KNN

In [0]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(anchor, newsgroups_train.target)
neight.predict(anchor_new)

# LSH

In [0]:
from sklearn.neighbors import LSHForest

lshf = LSHForest(random_state=42)
lshf.fit(train_loader)
LSHForest(min_hash_match=4, n_candidates=50, n_estimators=1,
          n_neighbors=5, radius=1.0, radius_cutoff_ratio=0.9,
          random_state=42)
distances, indices = lshf.kneighbors(X_test, n_neighbors=2)

# accuracy

In [0]:
# grid-search over hidden_dim and fc_num